## Forward Kinematics

In [2]:
from sympy import symbols, cos, sin, pi, simplify, sqrt, atan2
from sympy.matrices import Matrix
from numpy import array

import numpy as np

In [3]:
# Create symbols for joint variables
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8')
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8')

In [4]:
# DH parameter table
DH_table = {alpha0: 0, a0: 0, d1:0.75, 
            alpha1: -pi/2, a1: 0.35, d2: 0, q2: q2-pi/2,
            alpha2: 0, a2: 1.25, d3:0,
            alpha3: -pi/2, a3: -0.054, d4: 1.50,
            alpha4: pi/2, a4: 0, d5: 0,
            alpha5: -pi/2, a5: 0, d6: 0,
            alpha6: 0, a6: 0, d7: 0.303, q7: 0}

In [5]:
# Transformation matrix from frame {i-1} to {i}
def TF_matrix(alpha, a, d, q):
    return Matrix([
        [cos(q), -sin(q), 0, a],
        [sin(q)*cos(alpha), cos(q)*cos(alpha), -sin(alpha), -sin(alpha)*d],
        [sin(q)*sin(alpha), cos(q)*sin(alpha), cos(alpha), cos(alpha)*d],
        [0, 0, 0, 1]
    ])

In [6]:
# Individual transformation matrices
T0_1 = TF_matrix(alpha0, a0, d1, q1).subs(DH_table)
T1_2 = TF_matrix(alpha1, a1, d2, q2).subs(DH_table)
T2_3 = TF_matrix(alpha2, a2, d3, q3).subs(DH_table)
T3_4 = TF_matrix(alpha3, a3, d4, q4).subs(DH_table)
T4_5 = TF_matrix(alpha4, a4, d5, q5).subs(DH_table)
T5_6 = TF_matrix(alpha5, a5, d6, q6).subs(DH_table)
T6_G = TF_matrix(alpha6, a6, d7, q7).subs(DH_table)

In [14]:
T0_G = T0_1*T1_2*T2_3*T3_4*T4_5*T5_6*T6_G

In [15]:
print(T0_G.evalf(subs={q1:0, q2:0, q3:0, q4:0, q5:0, q6: 0, q7:0}))

Matrix([[0, 0, 1.00000000000000, 2.15300000000000], [0, -1.00000000000000, 0, 0], [1.00000000000000, 0, 0, 1.94600000000000], [0, 0, 0, 1.00000000000000]])


## Inverse kinematics

In [17]:
# Correction to gripper link orientation URDF vs DH Convention
yaw, pitch, roll = 0, 0, 0

R_z = Matrix([  
            [cos(q1), -sin(q1), 0],
            [sin(q1), cos(q1), 0],                                                                             
            [0, 0, 1],                                                                                         
            ])
R_y = Matrix([  
            [cos(q1), 0, sin(q1)],                                                                             
            [0, 1, 0],
            [-sin(q1), 0, cos(q1)],                                                                            
            ])
R_x = Matrix([  
            [1, 0, 0],
            [0, cos(q1), -sin(q1)],
            [0, sin(q1), cos(q1)],                                                                             
            ])                                                                                                 
R_corr = (R_z.subs({q1:pi}) * R_y.subs({q1:-pi/2}))

Rrpy = R_z.subs({q1:yaw}) * R_y.subs({q1:pitch}) * R_x.subs({q1:roll}) * R_corr

In [ ]:
# Vector along z-axis
nx, ny, nz = Rrpy[:,2] # z-axis

d = 0.303 # from URDF file

# Position of the wrist center
wx = px - d * nx
wy = py - d * ny
wz = pz - d * nz

            # Calculate joint angles using Geometric IK method
            theta1 = atan2(wy, wx)

            # magic numbers below from modified DH table
            side_a = 1.501 # sqrt(a3^2 + d4^2)
            side_b = sqrt(pow((sqrt(wx * wx + wy * wy) - 0.35), 2) + pow(wz - 0.75, 2)) # a1, d1
            side_c = 1.25 # a2

            # Law of cosines
            angle_a = acos((side_b * side_b + side_c * side_c - side_a * side_a) / (2 * side_b * side_c))
            angle_b = acos((side_a * side_a + side_c * side_c - side_b * side_b) / (2 * side_a * side_c))
            angle_c = acos((side_a * side_a + side_b * side_b - side_c * side_c) / (2 * side_a * side_b))

            # mind the offset
            theta2 = pi/2 - angle_a - atan2(wz - 0.75, sqrt(wx * wx + wy * wy) - 0.35) # d1, a1
            theta3 = pi/2 - (angle_b + 0.036) # abs(a3/d4) * dtr

            # Rotation matrix
            T0_3 = T0_1[:3,:3] * T1_2[:3,:3] * T2_3[:3,:3]
            R0_3 = T0_3.evalf(subs={q1: theta1, q2: theta2, q3: theta3})
            # somehow transpose() seems to work better than .inv("LU"), less redundant wrist rotations
            R3_6 = R0_3.transpose() * Rrpy

            # Euler angles from a Rotation matrix
            theta4 = atan2(R3_6[2,2], -R3_6[0,2])
            theta5 = atan2(sqrt(R3_6[0,2]*R3_6[0,2]+R3_6[2,2]*R3_6[2,2]),R3_6[1,2])
            theta6 = atan2(-R3_6[1,1], R3_6[1,0])
            ###
